**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

In [ ]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## <font color='blue'> Q2 [20 points total] </font>

### <font color='blue'> Implement a sponge hash function H with a 256 bit output that is based on AES-128 with a fixed key input as the permutation P. Assume c=112 most significant bits of the internal state are not modified by message blocks and H0=0xFFFF...FFFF is the initial input. Use proper padding for incomplete input blocks. Showcase your program by hashing the message "The Sponge construction is simpler than the Merkle-Damgard construction".

In [ ]:
from Crypto.Cipher import AES

In [3]:
from Cryptodome.Cipher import AES


In [4]:
# you can use this function to xor two bytes, or two strings.
def xor(x,y):
    if type(x) == type(y) == str:
        return "".join([chr(ord(a) ^ ord(b)) for a,b in zip(x,y)])
    elif type(x) == type(y) == bytes:
        return "".join([chr(a ^ b) for a,b in zip(x,y)]).encode('iso-8859-1')
    else:
        print("Error: type mismatch")

In [5]:
BLOCK_SIZE = 16
SPONGE_C_BYTES = 112//8

In [39]:
# 5 points
def sponge_digest(cipher, state: bytes, message: bytes) -> bytes:
    """ Implement the digest part of the hash function. The first
    112 bits of state should no be modified by message blocks. return
    the updated state in bytes format.
    """
    # YOUR CODE HERE
    r=BLOCK_SIZE*8-112 #rate 
    r=r//8 # convert to bytes
    for i in range(0,len(message),r):
        message_block=message[i:i+r] # message block
        if len(message_block)<r:
            pad_len=(r-len(message_block))*8 # get padding length
            pad='0'*(pad_len) 
            pad=int('1'+pad[1:],2) # add padding
            message_block += pad.to_bytes(pad_len//8,'big') # add to message block
        state=state[:-r]+xor(state[-r:],message_block) # update state
        state=cipher.encrypt(state) #encrypt state
    return state[-BLOCK_SIZE:]

# 5 points
def sponge_squeeze(cipher, state: bytes) -> bytes:
    """ Implement the squeeze part of the hash function """
    output = b""
    # YOUR CODE HERE
    r=BLOCK_SIZE*8-112
    r=r//8
    output += state[:r]
    #encrypt state 
    while len(output) < BLOCK_SIZE*2:
        state=cipher.encrypt(state)
        output += state[:r]
    
    return output

# 10 points
def sponge_hash(key: bytes, message: bytes) -> str:
    """ The sponge based hash function. The output should
    be in hex string format.
    """

    ouptut = b""
    # YOUR CODE HERE
    state=b'ff'*BLOCK_SIZE # initialise state
    cipher=AES.new(key,AES.MODE_ECB)
    state=sponge_digest(cipher,state,message) # absorb state
    output=sponge_squeeze(cipher,state) # squeeze state
    
    
    return output.hex()

In [40]:
state = b"ff" * BLOCK_SIZE
key = b"a5" * 16
cipher = AES.new(key, AES.MODE_ECB)
state = sponge_digest(cipher, state, b"This is a test message")
assert type(state) == bytes
assert state.hex() == "6aa4086babb6e210e5c51a06a318fe62"

In [41]:
state = b"ff" * BLOCK_SIZE
key = b"a5" * 16
cipher = AES.new(key, AES.MODE_ECB)
state = sponge_digest(cipher, state, b"This is a test message")
output = sponge_squeeze(cipher, state)
assert type(output) == bytes
assert output.hex() == "6aa4b33cce01b32b11460b444410e3765e9e80785b380786b7321f62c8ced76b"

In [42]:
msg = b"The Sponge construction is simpler than the Merkle-Damgard construction"
key = b"a5" * 16
hashval = sponge_hash(key, msg)
assert hashval == "83d4e1f3efff639d7113d880e8745ee18c5be5b5b68edcbea587a351e1752b09"